In [1]:
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
#import os
import seaborn as sns
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [157]:
data= pd.read_excel('Sales_project.xlsx')
sales_order= pd.read_excel('Sales_project.xlsx','Sales Orders Sheet')
customers=pd.read_excel('Sales_project.xlsx', 'Customers Sheet')
site_location= pd.read_excel('Sales_project.xlsx','Site Location sheet')
products= pd.read_excel('Sales_project.xlsx', 'Products_Sheet')
sales_team= pd.read_excel('Sales_project.xlsx','Sales Team Sheet')

In [158]:
sales_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7991 entries, 0 to 7990
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   OrderNumber       7991 non-null   object        
 1   OrderDate         7991 non-null   datetime64[ns]
 2   CurrencyCode      7991 non-null   object        
 3   _SalesAM_ID       7991 non-null   int64         
 4   _CustomerID       7991 non-null   int64         
 5   _SiteID           7991 non-null   int64         
 6   _ProductID        7991 non-null   int64         
 7   Order Quantity    7991 non-null   int64         
 8   Discount Applied  7991 non-null   float64       
 9   Unit Price        7991 non-null   float64       
 10  Unit Cost         7991 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 686.9+ KB


In [159]:
products.head(13)

,_ProductID,Product Name,Item/Service Description,Bandwidth
0,1,4G LTE connectivity,4G GO SMART 1GB/DAY,0
1,2,4G LTE connectivity,4G MUP 1GB/DAY GOV,0
2,3,4G LTE connectivity,4G GO PLATINUM 4GB/DAY,0
3,4,4G LTE connectivity,4G UNLIMITED 100GB,0
4,5,4G LTE connectivity,4G ONE NETWORK 3MBPS + INSTALLATION,0
5,6,4G LTE connectivity,4G 30GB Bundle,0
6,7,4G LTE connectivity,4G ONE NETWORK 5MBPS + DEVICE,0
7,8,COLOCATION,COLOCATION HALF RACK,0
8,9,Cloud Computing,Cloud 4vCPU. 8GB RAM. 300GB Storage,0
9,10,Cloud Computing,Cloud 12vCPU. 24GB RAM. 800GB Storage,0


In [56]:
products.loc[products.index == 45 ]

,_ProductID,Product Name,Item/Service Description,Bandwidth
45,I0565,Fiber Broadband Internet,FIBER INTERNET 10MBPS GOV,10


In [137]:
sales_order.head()

,OrderNumber,OrderDate,CurrencyCode,_SalesAM_ID,_CustomerID,_SiteID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000175,2018-06-09,RWF,1,1,107,1,1,0.075,2224.4,1579.324
1,SO - 0003171,2019-05-26,RWF,8,1,288,1,1,0.300,3966.4,2816.144
2,SO - 0003587,2019-07-16,RWF,11,1,129,1,2,0.100,3852.5,2658.225
3,SO - 0005097,2020-01-14,RWF,22,1,4,1,1,0.075,2505.8,1954.524
4,SO - 0005532,2020-03-04,RWF,11,1,48,1,1,0.100,3256.2,2767.770


# Data Augementation

In [160]:
#Drop the Currrency Column since it's universal
sales_order.drop(['CurrencyCode'], axis=1, inplace=True)

In [161]:
#Calculate total amount of each transaction with discount deduction
sales_order['Total Amount'] = (sales_order['Order Quantity']* sales_order['Unit Price'])-sales_order['Discount Applied']

In [162]:
#Create Month/Year column from datetime column
sales_order['Month'] = pd.DatetimeIndex(sales_order['OrderDate']).month
sales_order['Year'] = pd.DatetimeIndex(sales_order['OrderDate']).year

In [163]:
#Calculate profit from each individual sale transaction
sales_order['Profit'] = (sales_order['Unit Price']- sales_order['Unit Cost'])-sales_order['Discount Applied']

In [165]:
#Add Bandwidth purchased from each transaction
sales_order = sales_order.merge(products[['_ProductID','Bandwidth']], left_on = "_ProductID", right_on = "_ProductID", how = "left")

In [189]:
#Add last purchase data in days from
def last_purchase(customer_id, OrderIndex):
    customer_purchase = sales_order.query("_CustomerID == {customer_id}").sort_values(['OrderDate'])
    current_purchase_date = customer_purchase.loc[OrderIndex].OrderDate #current purchase data
    last_purchase_date = customer_purchase.loc[OrderIndex -1]
    

In [199]:
customer_purchase.iloc[0]

OrderNumber                 SO - 000169
OrderDate           2018-06-08 00:00:00
_SalesAM_ID                           9
_CustomerID                           1
_SiteID                              30
_ProductID                           35
Order Quantity                        2
Discount Applied                   0.15
Unit Price                       1051.9
Unit Cost                       494.393
Total Amount                    2103.65
Month                                 6
Year                               2018
Profit                          557.357
Bandwidth                             0
Name: 5799, dtype: object

In [202]:
#customer_purchase = sales_order.query("_CustomerID == 1").sort_values(['OrderDate'])
#current_purchase_date = customer_purchase.loc[0].OrderDate
last_purchase_date = customer_purchase.iloc[0 -1]

In [191]:
current_purchase_date

Timestamp('2018-06-09 00:00:00')

In [181]:
sales_order.query("_CustomerID == 1").sort_values(['OrderDate'])

,OrderNumber,OrderDate,_SalesAM_ID,_CustomerID,_SiteID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost,Total Amount,Month,Year,Profit,Bandwidth
5799,SO - 000169,2018-06-08,9,1,30,35,2,0.150,1051.9,494.393,2103.650,6,2018,557.357,0
0,SO - 000175,2018-06-09,1,1,107,1,1,0.075,2224.4,1579.324,2224.325,6,2018,645.001,0
1034,SO - 000307,2018-06-26,12,1,299,7,1,0.075,1078.7,506.989,1078.625,6,2018,571.636,0
5482,SO - 000368,2018-07-02,11,1,160,33,6,0.075,5098.7,3416.129,30592.125,7,2018,1682.496,0
4836,SO - 000400,2018-07-06,23,1,120,29,7,0.050,5708.4,2682.948,39958.750,7,2018,3025.402,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3438,SO - 0007591,2020-11-01,23,1,171,21,4,0.300,5246.1,3619.809,20984.100,11,2020,1625.991,50
6870,SO - 0007717,2020-11-16,6,1,240,41,4,0.300,1005.0,633.150,4019.700,11,2020,371.550,60
1736,SO - 0007807,2020-11-25,25,1,88,11,4,0.100,1815.7,1034.949,7262.700,11,2020,780.651,0
4837,SO - 0008018,2020-12-22,18,1,162,29,3,0.150,1943.0,1573.830,5828.850,12,2020,369.020,0


In [214]:
sales_order['Month-Year']= sales_order['OrderDate'].dt.strftime('%m/%Y')

In [232]:
#sales_order['_ProductID'].groupby([sales_order.Year]).agg('count')
sales_order['_ProductID'].groupby([sales_order["Month-Year"]]).agg('count')

Month-Year
01/2019    293
01/2020    276
02/2019    232
02/2020    270
03/2019    213
03/2020    236
04/2019    271
04/2020    262
05/2018      8
05/2019    279
05/2020    263
06/2018    252
06/2019    243
06/2020    245
07/2018    261
07/2019    253
07/2020    281
08/2018    279
08/2019    261
08/2020    251
09/2018    254
09/2019    237
09/2020    254
10/2018    228
10/2019    235
10/2020    282
11/2018    257
11/2019    262
11/2020    262
12/2018    297
12/2019    251
12/2020    243
Name: _ProductID, dtype: int64

In [215]:
sales_order.head()

,OrderNumber,OrderDate,_SalesAM_ID,_CustomerID,_SiteID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost,Total Amount,Month,Year,Profit,Bandwidth,Month-Year
0,SO - 000175,2018-06-09,1,1,107,1,1,0.075,2224.4,1579.324,2224.325,6,2018,645.001,0,06/2018
1,SO - 0003171,2019-05-26,8,1,288,1,1,0.300,3966.4,2816.144,3966.100,5,2019,1149.956,0,05/2019
2,SO - 0003587,2019-07-16,11,1,129,1,2,0.100,3852.5,2658.225,7704.900,7,2019,1194.175,0,07/2019
3,SO - 0005097,2020-01-14,22,1,4,1,1,0.075,2505.8,1954.524,2505.725,1,2020,551.201,0,01/2020
4,SO - 0005532,2020-03-04,11,1,48,1,1,0.100,3256.2,2767.770,3256.100,3,2020,488.330,0,03/2020


# Building the model

## Predicting future products demand

In [168]:
x = sales_order.drop(["_ProductID","OrderNumber","OrderDate","Year"], axis="columns")
y = sales_order["_ProductID"]

In [180]:
sales_order.head()

,OrderNumber,OrderDate,_SalesAM_ID,_CustomerID,_SiteID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost,Total Amount,Month,Year,Profit,Bandwidth
0,SO - 000175,2018-06-09,1,1,107,1,1,0.075,2224.4,1579.324,2224.325,6,2018,645.001,0
1,SO - 0003171,2019-05-26,8,1,288,1,1,0.300,3966.4,2816.144,3966.100,5,2019,1149.956,0
2,SO - 0003587,2019-07-16,11,1,129,1,2,0.100,3852.5,2658.225,7704.900,7,2019,1194.175,0
3,SO - 0005097,2020-01-14,22,1,4,1,1,0.075,2505.8,1954.524,2505.725,1,2020,551.201,0
4,SO - 0005532,2020-03-04,11,1,48,1,1,0.100,3256.2,2767.770,3256.100,3,2020,488.330,0


In [170]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)
X_train.dtypes

_SalesAM_ID           int64
_CustomerID           int64
_SiteID               int64
Order Quantity        int64
Discount Applied    float64
Unit Price          float64
Unit Cost           float64
Total Amount        float64
Month                 int64
Profit              float64
Bandwidth             int64
dtype: object

In [171]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [172]:
c =model.predict(X_test)

In [177]:
print("Accuracy:",model.score(X_test, y_test))

Accuracy: 0.2772772772772773


Accuracy: 1.0
